# Laboratorio 2
## Microestructuras y Sistemas de Trading
### Valentina Escobar y Pablo Mendoza

El presente laboratorio tiene como objetivo analizar el comportamiento de un ETF de acuerdo a la conformación de este y al movimiento de los precios que se presentaron entre el 31 de marzo del 2017 hasta el 29 de marzo del 2018 con el fin de identificar que tan conveniente es para un inversionista rebalancear diariamente durante un año comprando $n$ acciones cuando el rendimiento diario sea de -1%. Se contrastará el capital inicial con el capital final deshaciendo todas las pocisiones del portafolo el 1 de Marzo de 2018. Se realizan diferentes medidas de desempeño que permitan comparar los portafolios resultantes.

## Librerías y Funciones 

In [1]:
# Librerias
import quandl
import datetime
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
quandl.ApiConfig.api_key = "My9ygJsz7yTDx_9jcneH" # Quandl API key
fechas = list(['2017-03-31','2018-03-29'])

In [3]:
data_csv = pd.read_csv('/Users/pablomendoza/Documents/ITESO/Trading/Lab_1/Trading/Data/IJH_holdings-1.csv', skiprows = [0,1,2,3,4,5,6,7,8,9], index_col = ['Ticker'])
tick = list(data_csv.index)
weights = data_csv['Weight (%)']
#del data_csv
def Download_QuandL(fechas:'Dates_list ', tick:'Tickers of Stock'):
    dates = { 'gte': fechas[0], 'lte': fechas[1] }
    column = { 'columns': ['date', 'Open', 'Close']}
    data = quandl.get_table('WIKI/PRICES', qopts = column, ticker = tick, date = dates, paginate=True)# ticker = 'WIKI/AAPL' #'AAPL.US'
    data = data.iloc[::-1]
    data = data.reset_index()
    data = data.drop(['None'],1)
    return data
def All_Data(fechas:'Dates_list ', tick:'Ticker'):
    d = {} # Crea un Diccionario Vacío
    for i in range(len(tick)): # ciclo que descarga los datos de Quandl y los guarda como DataFrames en un Diccionario
        nom = str(tick[i])
        d["_{0}".format(nom)] = Download_QuandL(fechas, tick[i])
    return d
def Rebalanceo(df, Com, Cap_I, R_R, n_a, we):
    """
    Esta funcion hace los rebalanceos de cada activo de forma diaria por una año
    :param df: DataFrame del Activo
    :param Com: Comisión
    :param Cap_I: Capital Inicial
    :param R_R: Rendimiento de Rebalanceo
    :param n_a: Acciones que se compran al rebalancear
    :param we: 
    :return: 
    """
    a = (df['close']/df['open'])-1
    P_se = df['close']-df['open']
    add_s = list()
    Opp = list()
    comi = list()
    In_shares = np.floor((we*Cap_I)/df['close'][0])
    for i in range(len(df)):
        if a.iloc[i] < R_R:
            add_s.append(n_a)
            Opp.append('Compra')
            comi.append(Com*df['close'].iloc[i]*n_a)
        else:
            add_s.append(0)
            Opp.append('-')
            comi.append(0)
    df = df.join(pd.DataFrame(add_s, columns = ['# Added Shares']))
    df = df.join(pd.DataFrame(Opp, columns = ['Operacion']))
    df = df.join(pd.DataFrame(comi, columns = ['Comisión']))
    df = df.join(pd.DataFrame(P_se, columns = ['Precio de Señal %']))
    Tot_Acciones = pd.DataFrame(list(df['# Added Shares'].cumsum()), columns = ['Shares'])+In_shares
    df = df.join(Tot_Acciones)
    df.head()
    return df
def Clean_Dicc(dic:'Diccionario a limpiar',sizes:'Tamaño deseado del DF'):
    y = list(dic.keys())
    for h in y:
        if len(dic[h])!=248:
            del dic[h]
    return dic

In [4]:
Data = All_Data(fechas, tick)

In [5]:
d_full = Clean_Dicc(Data,248)

In [6]:
tick = list(d_full.keys())
d2 = {}
for j in range(len(tick)):
    tabla = Rebalanceo(d_full[tick[j]],0.0003,1000000,-0.01,10,weights[j])
    d2["Data{0}".format(j)] = tabla

In [7]:
d2['Data0']

,date,open,close,# Added Shares,Operacion,Comisión,Precio de Señal %,Shares
0,2017-03-31,71.32,71.97,0,-,0.00000,0.65,8475.0
1,2017-04-03,71.65,71.38,0,-,0.00000,-0.27,8475.0
2,2017-04-04,71.38,70.55,10,Compra,0.21165,-0.83,8485.0
3,2017-04-05,70.45,69.26,10,Compra,0.20778,-1.19,8495.0
4,2017-04-06,69.25,69.05,0,-,0.00000,-0.20,8495.0
5,2017-04-07,68.85,69.03,0,-,0.00000,0.18,8495.0
6,2017-04-10,69.08,68.98,0,-,0.00000,-0.10,8495.0
7,2017-04-11,68.97,69.43,0,-,0.00000,0.46,8495.0
8,2017-04-12,67.86,68.41,0,-,0.00000,0.55,8495.0
9,2017-04-13,68.19,68.12,0,-,0.00000,-0.07,8495.0
